In [1]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('wikipedia_dataset_hasan/wikipedia.csv')

In [2]:
titles = pd.DataFrame(dataset, columns = ['page_title']).head(5)

titles.to_csv('wikipages.txt', index=None, header=None)

titles

,page_title
0,Casino Royale (2006 film)
1,Procellariidae
2,Kakapo
3,2005 NFL Draft
4,Danish football champions


In [3]:
import requests

session = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

title = "Procellariidae"

PARAMS = {
    "action": "query",
    "prop": "revisions",
    "format": "json",
    "titles": title,
    "rvlimit":50,
    "rvstart": "2017-04-03T01:32:59.000Z",
    "rvdir": "older",
}

data = session.get(url=URL, params=PARAMS).json()

data

{'continue': {'rvcontinue': '20120325061631|483806763', 'continue': '||'},
 'query': {'pages': {'224443': {'pageid': 224443,
    'ns': 0,
    'title': 'Procellariidae',
    'revisions': [{'revid': 755018716,
      'parentid': 755018492,
      'user': 'Pvmoutside',
      'timestamp': '2016-12-15T20:26:53Z',
      'comment': ''},
     {'revid': 755018492,
      'parentid': 755018361,
      'user': 'Pvmoutside',
      'timestamp': '2016-12-15T20:25:25Z',
      'comment': ''},
     {'revid': 755018361,
      'parentid': 753565639,
      'user': 'Pvmoutside',
      'timestamp': '2016-12-15T20:24:41Z',
      'comment': ''},
     {'revid': 753565639,
      'parentid': 742796045,
      'user': '128.227.180.135',
      'anon': '',
      'timestamp': '2016-12-07T22:57:53Z',
      'comment': ''},
     {'revid': 742796045,
      'parentid': 732146237,
      'user': 'Od Mishehu',
      'timestamp': '2016-10-05T21:06:58Z',
      'comment': '{{[[template:Automatic Taxobox|Automatic Taxobox]]}}'},
   

In [4]:
from pandas.io.json import json_normalize

#revisions = data["query"]["pages"][page_id]["revisions"]
revisions = list(data["query"]["pages"].values())[0]["revisions"]

json_normalize(revisions)

,anon,comment,minor,parentid,revid,timestamp,user
0,NaN,,NaN,755018492,755018716,2016-12-15T20:26:53Z,Pvmoutside
1,NaN,,NaN,755018361,755018492,2016-12-15T20:25:25Z,Pvmoutside
2,NaN,,NaN,753565639,755018361,2016-12-15T20:24:41Z,Pvmoutside
3,,,NaN,742796045,753565639,2016-12-07T22:57:53Z,128.227.180.135
4,NaN,{{[[template:Automatic Taxobox|Automatic Taxob...,NaN,732146237,742796045,2016-10-05T21:06:58Z,Od Mishehu
5,NaN,/* External links */ Repos. navbar.,NaN,729804432,732146237,2016-07-29T23:21:59Z,Thor Dockweiler
6,NaN,/* External links */ Add navbar + repos. ACs.,NaN,725860152,729804432,2016-07-14T17:32:36Z,Thor Dockweiler
7,NaN,remove dead link where permanent doi present,NaN,724430501,725860152,2016-06-18T09:48:47Z,DrKay
8,NaN,[[User:Green Cardamom/WaybackMedic|WaybackMedic]],NaN,723779107,724430501,2016-06-09T06:27:49Z,GreenC bot
9,NaN,"refs, [[WP:AWB/T|typo(s) fixed]]: between 3 to...",,717842921,723779107,2016-06-05T05:34:06Z,Dcirovic


In [6]:
import requests

S = requests.Session()
URL = "https://en.wikipedia.org/w/api.php"

def request_page(title, date):
    # title = "Procellariidae"
    # date = "2017-04-03T01:32:59.000Z"

    params = {
        "action": "query",
        "prop": "revisions",
        "titles": title,
        "rvprop": "timestamp|user|comment|content",
        "rvslots": "main",
        "formatversion": "2",
        "format": "json",
        "rvlimit":1,
        "rvstart": date,
        "rvdir": "older",
    }

    response = S.get(url=URL, params=params)
    return response.json()

request_page("Procellariidae", "2017-04-03T01:32:59.000Z")



{'continue': {'rvcontinue': '20161215202525|755018492', 'continue': '||'},
 'query': {'pages': [{'pageid': 224443,
    'ns': 0,
    'title': 'Procellariidae',
    'revisions': [{'user': 'Pvmoutside',
      'timestamp': '2016-12-15T20:26:53Z',
      'slots': {'main': {'contentmodel': 'wikitext',
        'contentformat': 'text/x-wiki',
        'content': '{{pp-move-indef}}\n{{Automatic Taxobox\n| name = Procellariidae\n| image =Damier du Cap - Cape Petrel.jpg\n| image_width =\n| image_caption = [[Cape petrel]] (\'\'Daption capense\'\')\n| taxon = Procellariidae\n| diversity_ref = <ref name=Taxonomicon/>\n| diversity = 16 genera and about 70 species\n| diversity_link = List of Procellariidae\n| authority = [[William Elford Leach|Leach]], 1820<ref name=Taxonomicon>{{cite web| url=http://taxonomicon.taxonomy.nl/TaxonTree.aspx?id=51489&tree=0.1 | title=Systema Naturae 2000 / Classification&nbsp;— Superfamily Procellarioidea | accessdate=12 Feb 2009 | last=Brands| first=Sheila | date=Aug 14, 